<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E4%B8%AD%E6%96%87%E7%AC%AC%E4%B8%80%E6%AC%A1%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import os
import httpx

os.environ["OPENAI_API_KEY"]  = "sk-JEB"

import openai
from openai import AsyncOpenAI


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Wuxia/input/all_direct

dialogueful_novel.jsonl  part_10.txt  part_2.txt  part_4.txt  part_6.txt  part_8.txt
part_0.txt		 part_1.txt   part_3.txt  part_5.txt  part_7.txt  part_9.txt


In [ ]:
def count_quotes(text):

    quote_chars = '''「」"“”'''
    count = 0
    for char in text:
        if char in quote_chars:
            count += 1
    return count

In [ ]:
from tqdm import tqdm

novel2count = {}

source_pool = set()

import json

datas = []
for part_id in tqdm(range(0, 10)):
    fname = "/content/drive/MyDrive/Wuxia/input/all_direct/part_"+ str(part_id) +".txt"


    with open(fname, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip() == "":
                continue
            data = json.loads(line)
            source = data["source"]
            if source in source_pool:
                continue

            if source not in novel2count:
                novel2count[source] = 0

            if novel2count[source] > 100:
                continue

            n_quote = count_quotes(data["text"])
            if n_quote < 100:
                continue

            novel2count[source] += 1
            datas.append(data)

    print(len(datas))

 10%|█         | 1/10 [00:39<05:51, 39.04s/it]

3326


 20%|██        | 2/10 [01:12<04:47, 35.97s/it]

7150


 30%|███       | 3/10 [01:48<04:11, 36.00s/it]

...

26417


 90%|█████████ | 9/10 [05:18<00:35, 35.43s/it]

29384


100%|██████████| 10/10 [05:58<00:00, 35.80s/it]

32732


In [ ]:
raw_datas = datas

In [ ]:
print(datas[100]['text'])

给定Paragraph，抽取其中的对话，并输出为json格式

Let's think it step by step
1. 对Paragraph进行总结，存储在summary字段
2. 抽取每一句对话的内容 dialogue，以及对话的说话人 said by, 存储在conversations中



In [ ]:
task_prompt = """给定input paragraph，抽取其中的对话，并输出为json格式

Let's think it step by step
1. summarize input paragraph into bullet format，存储在summary字段
2. 抽取每一句对话的内容 dialogue，判断每一句话的说话人 said by, 存储在conversations中"""

example_input = """input paragraph:
“你认为明天的天气怎么样？”，来人是一个女士。李回答说：“Alice，据天气预报，明天会下雨。”Alice叹了口气，继续说：“那我们的野餐计划可能要取消了。”"""

example_output = {
  "summary": "- Alice和李在公园讨论明天的天气和他们的野餐计划。",
  "conversations": [
    {
      "dialogue": "你认为明天的天气怎么样？",
      "said_by": "Alice"
    },
    {
      "dialogue": "Alice，据天气预报，明天会下雨。",
      "said_by": "李"
    },
    {
      "dialogue": "那我们的野餐计划可能要取消了。",
      "said_by": "Alice"
    }
  ]
}

long_task_prompt = """给定input paragraph，抽取其中的对话，并输出为json格式

Let's think it step by step
1. summarize input paragraph into bullet format，存储在summary字段
2. 抽取每一句对话的内容 dialogue，判断每一句话的说话人 said by, 存储在conversations中

Example input paragraph:
“你认为明天的天气怎么样？”，来人是一个女士。李回答说：“Alice，据天气预报，明天会下雨。”Alice叹了口气，继续说：“那我们的野餐计划可能要取消了。”

Example Output:
"""

import json
example_output_str = json.dumps(example_output, ensure_ascii=False,indent = 4)
# print(example_output_str)

long_task_prompt += example_output_str

long_task_prompt = """给定input paragraph，抽取其中的对话，并输出为json格式

Let's think it step by step
1. summarize input paragraph into bullet format，存储在summary字段
2. 抽取每一句对话的内容 dialogue，判断每一句话的说话人 said by, 存储在conversations中

Example input paragraph:
许媚娘怒瞪着杨浩，好半响，她才说道：“你连姑姑的话都敢不听？”
杨浩汗啊，这都什么跟什么？这许媚娘也太自以为是了吧？
“有个问题我不明白，你既是龙家的人，为什么就不姓龙呢？”杨浩丝毫没有把许媚娘的怒意放在眼里，美人火，还是别有一番味道的。
“老爷子对我有恩，我就给他做女儿了，正好他也没有女儿。”许媚娘说道。
“哦，那请问你贵庚？”杨浩接着问道。
“小子，你不想混了是不是？姑姑的芳龄是你能随便问的吗？”许媚娘被气得不轻，以她的聪明，当然知道杨浩是想借故转移话题。
杨浩相当的无语，这女人，脸皮真厚，开口闭口都是以他的姑姑自居，不知道的人还真的以为她是他的姑姑呢。
只不过，杨浩随后又想到，好象还真的是那么回事，名义上来说，这个许媚娘真的是他姑姑。
“我是不会认他们的，他们也没有这个资格，你回去告诉他们，我现在对自己的生活很满意，不想打破现在的生活，让他们以后不要再来烦我。”杨浩说道。
“小宝贝，我真不知道你心里在想什么，龙家有什么不好？他们有什么不好？为什么你就不肯认回他们？”许媚娘就想明白杨浩是怎么想的。
“等等，你刚才喊我什么？”杨浩真不敢相信，他这么一个大老爷们的，被人喊什么小宝贝？

Example Output:
{
    "summary": "- 许媚娘对杨浩的行为表现出愤怒和不满。\n- 杨浩对许媚娘的态度感到困惑，同时试图转移话题。\n- 许媚娘坚称自己是杨浩的姑姑，而杨浩则对此表示不满。",
    "conversations": [
        {
            "dialogue": "你连姑姑的话都敢不听？",
            "said_by": "许媚娘"
        },
        {
            "dialogue": "有个问题我不明白，你既是龙家的人，为什么就不姓龙呢？",
            "said_by": "杨浩"
        },
        {
            "dialogue": "老爷子对我有恩，我就给他做女儿了，正好他也没有女儿。",
            "said_by": "许媚娘"
        },
        {
            "dialogue": "哦，那请问你贵庚？",
            "said_by": "杨浩"
        },
        {
            "dialogue": "小子，你不想混了是不是？姑姑的芳龄是你能随便问的吗？",
            "said_by": "许媚娘"
        },
        {
            "dialogue": "我是不会认他们的，他们也没有这个资格，你回去告诉他们，我现在对自己的生活很满意，不想打破现在的生活，让他们以后不要再来烦我。",
            "said_by": "杨浩"
        },
        {
            "dialogue": "小宝贝，我真不知道你心里在想什么，龙家有什么不好？他们有什么不好？为什么你就不肯认回他们？",
            "said_by": "许媚娘"
        },
        {
            "dialogue": "等等，你刚才喊我什么？",
            "said_by": "杨浩"
        }
    ]
}
"""


In [ ]:
!pip install -q aiofiles tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import os
import json
import time
# import openai
from openai import AsyncOpenAI

aclient = AsyncOpenAI()

import asyncio
import aiofiles
import tiktoken
import hashlib
# from connector import AsyncPGConnector
from tqdm.asyncio import tqdm as tqdm

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
en2zh_ratio = 2.3

delay = 1
concurrency_limit = 16

max_file_size = 1024**3

In [ ]:
for i, data in enumerate(datas):
    data['id'] = str(i)

print(datas[0].keys())

dict_keys(['source', 'text', 'id'])


In [ ]:
async def dealing_messages(messages):
    try:
        # request_token = sum([len(enc.encode(msg['content'])) for msg in messages])
        # response_token = int(len(enc.encode(text)) * en2zh_ratio) + 64

        model = "gpt-3.5-turbo-1106"

        resp = await aclient.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
            response_format={ "type": "json_object" }
        )

        try:
            result = resp.choices[0].message.content
            result = result.strip()
            return result
        except json.JSONDecodeError:
            print("Invalid json: ", result)
            return None
        except:
            raise Exception(f"Invalid API response: {resp}")

    except Exception as e:
        print(f"[Error] {e}")
        return None

def data2messages( data ):
    messages = [
        {"role":"system","content":task_prompt},
        {"role":"user","content":example_input},
        {"role":"assistant","content":example_output_str},
        {"role":"user","content":f"input paragraph:\n{data['text']}"}
    ]
    return messages

def data2messages_no_example( data ):
    messages = [
        {"role":"system","content":long_task_prompt},
        {"role":"user","content":f"input paragraph:\n{data['text']}"}
    ]
    return messages

In [ ]:
# messages = data2messages_no_example(datas[1])
# response = await dealing_messages(messages)
# print(response)

In [ ]:
async def getTranslation(item):
    if "messages" not in item:
        return None
    else:
        for i in range(3):
            result = await dealing_messages(item['messages'])
            if result is not None:
                item["response"] = result
                return item
            else:
                continue

    return None

async def process(item, semaphore):
    async with semaphore:
        try:
            output_folder = "/content/output"
            if "output_folder" in item:
                output_folder = item["output_folder"]
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            file_path = os.path.join(output_folder, f"{item['id']}.txt")

            if os.path.exists(file_path):
                return

            it = await getTranslation(item)
            if it is None:
                raise Exception(item['id'])

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(it, f, ensure_ascii=False)

        except Exception as e:
            print(f"Error processing entry: {e}")

In [ ]:
!rm -rf /content/output

In [ ]:
print(len(datas[0]['text']))

2898


In [ ]:
def data2messages_no_example_with_short_input( data ):
    text = data['text']
    lines = text.split('\n')
    n_token = 0
    paragraph = ""
    for line in lines:
        token_len = len( enc.encode(line) )
        if token_len + n_token > 1200:
            break
        n_token += token_len
        paragraph += line + '\n'

    if n_token < 600:
        paragraph = text[:600]
    paragraph = paragraph.strip()

    messages = [
        {"role":"system","content":long_task_prompt},
        {"role":"user","content":f"input paragraph:\n{paragraph}"}
    ]
    return messages

In [ ]:
async def main(datas):

    output_folder = "/content/output"

    process_data = []

    for data in datas:
        id = data['id']
        process_data.append({
            "id": id,
            "messages": data2messages_no_example_with_short_input(data),
            "output_folder": output_folder
        })

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # # print(f"Already processed {len(exist_list)} items...")

    # id = set()

    for item in process_data:
        file_path = os.path.join(output_folder, f"{item['id']}.txt")

        if os.path.exists(file_path):
            continue

        tasks.append(asyncio.create_task(process(item, semaphore)))

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)



In [ ]:
!rm -rf /content/output

In [ ]:
!ls /content/drive/MyDrive/CardBuild/dialogue_extract

In [ ]:
start_id = 10
end_id = 20

current_tasks = datas[start_id:end_id]

await main(current_tasks)

temp_output_folder = "/content/output"

for id in range(start_id, end_id):
    file_path = os.path.join(temp_output_folder, f"{id}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                response = data["response"]
            except:
                continue

        if response is not None:
            datas[id]["response"] = response
            datas[id]["input"] = data["messages"][1]["content"]
    # break

final_save_name = "/content/drive/MyDrive/CardBuild/dialogue_extract/" + str(start_id) + "_to_" + str(end_id) + ".txt"

with open(final_save_name, 'w', encoding='utf-8') as f:
    for id in range(start_id, end_id):
        json.dump(datas[id], f, ensure_ascii=False)
        f.write('\n')

Processing items: 100%|██████████| 10/10 [00:29<00:00,  2.96s/it]


In [ ]:
print(data)

{'id': '19', 'messages': [{'role': 'system', 'content': '给定input paragraph，抽取其中的对话，并输出为json格式\n\nLet\'s think it step by step\n1. summarize input paragraph into bullet format，存储在summary字段\n2. 抽取每一句对话的内容 dialogue，判断每一句话的说话人 said by, 存储在conversations中\n\nExample input paragraph:\n许媚娘怒瞪着杨浩，好半响，她才说道：“你连姑姑的话都敢不听？”\n杨浩汗啊，这都什么跟什么？这许媚娘也太自以为是了吧？\n“有个问题我不明白，你既是龙家的人，为什么就不姓龙呢？”杨浩丝毫没有把许媚娘的怒意放在眼里，美人火，还是别有一番味道的。\n“老爷子对我有恩，我就给他做女儿了，正好他也没有女儿。”许媚娘说道。\n“哦，那请问你贵庚？”杨浩接着问道。\n“小子，你不想混了是不是？姑姑的芳龄是你能随便问的吗？”许媚娘被气得不轻，以她的聪明，当然知道杨浩是想借故转移话题。\n杨浩相当的无语，这女人，脸皮真厚，开口闭口都是以他的姑姑自居，不知道的人还真的以为她是他的姑姑呢。\n只不过，杨浩随后又想到，好象还真的是那么回事，名义上来说，这个许媚娘真的是他姑姑。\n“我是不会认他们的，他们也没有这个资格，你回去告诉他们，我现在对自己的生活很满意，不想打破现在的生活，让他们以后不要再来烦我。”杨浩说道。\n“小宝贝，我真不知道你心里在想什么，龙家有什么不好？他们有什么不好？为什么你就不肯认回他们？”许媚娘就想明白杨浩是怎么想的。\n“等等，你刚才喊我什么？”杨浩真不敢相信，他这么一个大老爷们的，被人喊什么小宝贝？\n\nExample Output:\n{\n    "summary": "- 许媚娘对杨浩的行为表现出愤怒和不满。\n- 杨浩对许媚娘的态度感到困惑，同时试图转移话题。\n- 许媚娘坚称自己是杨浩的姑姑，而杨浩则对此表示不满。",\n    "conversations": [\n        {\n            "dialogue": "你连姑姑的话都敢不听？",\n            "said

In [ ]:
super_start = 20
super_end = len(datas)

n_bag = 50

# 计算每个子区间的长度
interval_length = (super_end - super_start) // n_bag

# 生成元组列表
se_tuples_list = [(super_start + i * interval_length, super_start + (i + 1) * interval_length) for i in range(n_bag - 1)]

# 添加最后一个元组，确保最后一个end等于super_end
se_tuples_list.append((se_tuples_list[-1][1], super_end))

# 打印生成的元组列表
print(se_tuples_list[0:2])
print(se_tuples_list[-2:])

import random


random.shuffle(se_tuples_list)

print(se_tuples_list[0])

[(20, 674), (674, 1328)]
[(31412, 32066), (32066, 32732)]
(20, 674)


In [ ]:
from IPython.display import clear_output

for start_id, end_id in se_tuples_list:
    current_tasks = datas[start_id:end_id]

    await main(current_tasks)
    await main(current_tasks)

    temp_output_folder = "/content/output"

    for id in range(start_id, end_id):
        file_path = os.path.join(temp_output_folder, f"{id}.txt")
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    response = data["response"]
                except:
                    continue

            if response is not None:
                datas[id]["response"] = response
        # break

    final_save_name = "/content/drive/MyDrive/CardBuild/dialogue_extract/" + str(start_id) + "_to_" + str(end_id) + ".txt"

    with open(final_save_name, 'w', encoding='utf-8') as f:
        for id in range(start_id, end_id):
            json.dump(datas[id], f, ensure_ascii=False)
            f.write('\n')

Processing items: 100%|██████████| 654/654 [19:04<00:00,  1.75s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [23:42<00:00,  2.18s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [28:41<00:00,  2.63s/it]
Processing items: 0it [00:00, ?it/s]
Processing items:   6%|▌         | 37/654 [01:03<10:22,  1.01s/it]

[Error] <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>


Processing items: 100%|██████████| 654/654 [32:12<00:00,  2.96s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [27:46<00:00,  2.55s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [20:08<00:00,  1.85s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [19:39<00:00,  1.80s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [24:53<00:00,  2.28s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [18:56<00:00,  1.74s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [20:19<00:00,  1.86s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [18:02<00:00,  1.65s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [19:11<00:00,  1.76s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 654/654 [18:57<00:00